<a href="https://www.kaggle.com/code/shivampandey06/gpt-model?scriptVersionId=170673474" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#for any gpt model to work we need data for its training. Here i am taking a Gutenburg data of a book called "A Book About ExtraordinaryManifestations of Nature"

!wget https://www.gutenberg.org/cache/epub/67223/pg67223.txt

--2024-04-06 17:15:26--  https://www.gutenberg.org/cache/epub/67223/pg67223.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 568188 (555K) [text/plain]
Saving to: 'pg67223.txt'

pg67223.txt         100%[===================>] 554.87K  1.79MB/s    in 0.3s    

2024-04-06 17:15:26 (1.79 MB/s) - 'pg67223.txt' saved [568188/568188]



Here clearly with the output , you can see we have downloaded and Uploaded the data, which will be used for the training purpose of the model. The uploaded file can be viewed on Output toggle on right.

In [2]:
#reading it in to inspect it
with open('/kaggle/working/pg67223.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))     
#this will return the number of the Characters in the File

length of dataset in characters:  554348


In [4]:
# let's look at the first 500 characters
print(text[:500])

﻿The Project Gutenberg eBook of The Strangest Things in the World: A Book About Extraordinary Manifestations of Nature
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the 


In [5]:
#This will return all the unique characters present in the text file
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !#$%'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyzóö—‘’“”• ™﻿
93


It shows that the file has numbers, special characters, small and capital English alphabets etc.

In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[63, 64, 64, 1, 75, 63, 60, 73, 60]
hii there


In [7]:
#let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500]) # the 500 characters we looked at earier will to the GPT look like this

torch.Size([554348]) torch.int64
tensor([92, 46, 63, 60,  1, 42, 73, 70, 65, 60, 58, 75,  1, 33, 76, 75, 60, 69,
        57, 60, 73, 62,  1, 60, 28, 70, 70, 66,  1, 70, 61,  1, 46, 63, 60,  1,
        45, 75, 73, 56, 69, 62, 60, 74, 75,  1, 46, 63, 64, 69, 62, 74,  1, 64,
        69,  1, 75, 63, 60,  1, 49, 70, 73, 67, 59, 24,  1, 27,  1, 28, 70, 70,
        66,  1, 27, 57, 70, 76, 75,  1, 31, 79, 75, 73, 56, 70, 73, 59, 64, 69,
        56, 73, 80,  1, 39, 56, 69, 64, 61, 60, 74, 75, 56, 75, 64, 70, 69, 74,
         1, 70, 61,  1, 40, 56, 75, 76, 73, 60,  0,  1,  1,  1,  1,  0, 46, 63,
        64, 74,  1, 60, 57, 70, 70, 66,  1, 64, 74,  1, 61, 70, 73,  1, 75, 63,
        60,  1, 76, 74, 60,  1, 70, 61,  1, 56, 69, 80, 70, 69, 60,  1, 56, 69,
        80, 78, 63, 60, 73, 60,  1, 64, 69,  1, 75, 63, 60,  1, 47, 69, 64, 75,
        60, 59,  1, 45, 75, 56, 75, 60, 74,  1, 56, 69, 59,  0, 68, 70, 74, 75,
         1, 70, 75, 63, 60, 73,  1, 71, 56, 73, 75, 74,  1, 70, 61,  1, 75, 63,
       

Here we have encoded the entire text into pytorch function. Pytorch is a python library used for building these function more can be readed at " ". as of Now, just remeber as we will be using this library in the code ahead.

In [8]:
# Let's now split up the data into train and validation sets
n = int(0.92*len(data)) # first 92% will be train, rest val
train_data = data[:n]
val_data = data[n:]

The built rule is for (90-10) i have used 92-8 , for better accuracy.

In [9]:
block_size = 8  # the block size don't have to do anything with my splitted data, you canm even choose any block size as well
train_data[:block_size+1]

tensor([92, 46, 63, 60,  1, 42, 73, 70, 65])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([92]) the target: 46
when input is tensor([92, 46]) the target: 63
when input is tensor([92, 46, 63]) the target: 60
when input is tensor([92, 46, 63, 60]) the target: 1
when input is tensor([92, 46, 63, 60,  1]) the target: 42
when input is tensor([92, 46, 63, 60,  1, 42]) the target: 73
when input is tensor([92, 46, 63, 60,  1, 42, 73]) the target: 70
when input is tensor([92, 46, 63, 60,  1, 42, 73, 70]) the target: 65


here, if you could analyse the output, we are just trying to predict the next word

In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[74, 58, 60, 69, 75,  1, 74, 71],
        [73, 59, 74,  1, 70, 61,  1, 75],
        [ 1, 78, 70, 70, 59, 67, 56, 69],
        [67, 70, 58, 66, 74, 12,  0,  0]])
targets:
torch.Size([4, 8])
tensor([[58, 60, 69, 75,  1, 74, 71, 70],
        [59, 74,  1, 70, 61,  1, 75, 63],
        [78, 70, 70, 59, 67, 56, 69, 59],
        [70, 58, 66, 74, 12,  0,  0, 87]])
----
when input is [74] the target: 58
when input is [74, 58] the target: 60
when input is [74, 58, 60] the target: 69
when input is [74, 58, 60, 69] the target: 75
when input is [74, 58, 60, 69, 75] the target: 1
when input is [74, 58, 60, 69, 75, 1] the target: 74
when input is [74, 58, 60, 69, 75, 1, 74] the target: 71
when input is [74, 58, 60, 69, 75, 1, 74, 71] the target: 70
when input is [73] the target: 59
when input is [73, 59] the target: 74
when input is [73, 59, 74] the target: 1
when input is [73, 59, 74, 1] the target: 70
when input is [73, 59, 74, 1, 70] the target: 61
when input is [

In [12]:
print(xb) # our input to the transformer

tensor([[74, 58, 60, 69, 75,  1, 74, 71],
        [73, 59, 74,  1, 70, 61,  1, 75],
        [ 1, 78, 70, 70, 59, 67, 56, 69],
        [67, 70, 58, 66, 74, 12,  0,  0]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 93])
tensor(5.1141, grad_fn=<NllLossBackward0>)

yGs)™_INzzO',Mó/n(A“[)t?Hsöq_sQ’K™PA7T*n_2W.A?
4LVnOxx‘srF
yUpm4ivXH#C krlOM]—su 0ó8RQgiqö*m8odUnvö:


In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

PyTorch Optimizer is a tool that helps with the process of training a machine learning model.Optimizers in PyTorch are responsible for updating the weights of a neural network during training.

In [15]:
batch_size = 32
for steps in range(120): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.915495872497559


In [16]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


I?ó*C‘j)[(czmV
g‘﻿yOJ5:0”xRga.g/1Hvf$ u[ynKY-UöqtT6U cr*/Dóe1kIocc5;Tf kX4RYNHR46-PC2x7B*WHSU6 tJDóF’5.S]I_%Y*EM’;hW4#t9dxód/,. Frs'Vd]l;Tb6FJWAIo _;w3_f5Se*7mGXcE0”nJfBD7uRDu!#RDh3
qbRgap5VnA“26A_?Xj)™_lJFF“Vw1$﻿4ó™•_y!Y-#RUi6P]—™c6d-JönA“6zz3aZwA‘d.rej(0﻿tuH’5/h uOKYBAóxlKH*!B(T’y3nR)b6b,f2zUCoNö2x‘XHm0e*!7[Kó*3'CnK3$k4%lrön,OK.‘%l‘sw1Wr*RseMCv%hlR—Z!$WcIPxp$2z™aGUqD3bBNlmqc!T
v3Kgf”cD™8Jl;FOAvWH'czzs5LphCq_/b?G.bMLtbR:?8),G Sl,löA ls f‘r8gPnK,T5,™”()L1voW!kBqINrqi.b”XH0”•Or8R7tJoSC?gaTIN?#ó! 


**Below is The mathematical trick in self-attention**  
For the pdf of the paper on self attention you can ping me, although it's a famous publication you can find it online.

In [17]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [18]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 #batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [19]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [20]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [21]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [22]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [23]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
Each example across batch dimension is of course processed completely independently and never "talk" to each other
In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
"self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
"Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [24]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5


In [25]:
k.var()

tensor(1.0449)

In [26]:
q.var()

tensor(1.0700)

In [27]:
wei.var()

tensor(1.0918)

In [28]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [29]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [30]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [31]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [32]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

This all was a Gpt code for the refrence in Packets
**You may find full finsihed code below**

In [33]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://www.gutenberg.org/cache/epub/67223/pg67223.txt
with open('/kaggle/working/pg67223.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.213341 M parameters
step 0: train loss 4.6399, val loss 4.6405
step 100: train loss 2.6349, val loss 2.8474
step 200: train loss 2.5131, val loss 2.7588
step 300: train loss 2.4411, val loss 2.7066
step 400: train loss 2.3607, val loss 2.5990
step 500: train loss 2.2964, val loss 2.5501
step 600: train loss 2.2484, val loss 2.5212
step 700: train loss 2.1989, val loss 2.4927
step 800: train loss 2.1450, val loss 2.4430
step 900: train loss 2.1215, val loss 2.3967
step 1000: train loss 2.0867, val loss 2.3460
step 1100: train loss 2.0480, val loss 2.3540
step 1200: train loss 2.0349, val loss 2.3259
step 1300: train loss 1.9997, val loss 2.2336
step 1400: train loss 1.9742, val loss 2.2523
step 1500: train loss 1.9595, val loss 2.2165
step 1600: train loss 1.9355, val loss 2.2041
step 1700: train loss 1.9154, val loss 2.1937
step 1800: train loss 1.9034, val loss 2.1834
step 1900: train loss 1.8821, val loss 2.1633
step 2000: train loss 1.8816, val loss 2.1636
step 2100: train loss 1.